In [1]:
import os
os.chdir("/work/pi_hzhang2_umass_edu/jnainani_umass_edu/diffusion_policy_interp/")
from sae.sae_model import SparseAutoencoder
import torch 
import pandas as pd 
from tqdm.auto import tqdm
from skvideo.io import vwrite
from IPython.display import Video
from diffusion_policy.policy.diffusion_transformer_lowdim_policy import DiffusionTransformerLowdimPolicy
from diffusion_policy.model.diffusion.transformer_for_diffusion import TransformerForDiffusion
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import collections
import random
from diffusion_policy.env.pusht.pusht_keypoints_env import PushTKeypointsEnv
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

seed1_acts = pd.read_csv("data/activations_summary.csv")
seed2_acts = pd.read_csv("data/activations_summary_2.csv")

sae_path = "sae/results_layer4_dim2048_k64_auxk64_dead200/checkpoints/last.ckpt"
sae_weights = torch.load(sae_path)
ckpt = {}
for k in sae_weights['state_dict'].keys():
    if k.startswith('sae_model.'):
        ckpt[k.split(".")[1]] = sae_weights['state_dict'][k]
sae = SparseAutoencoder(256, 2048, 64, 64, 32, 200)
sae.load_state_dict(ckpt)
sae.to(device)

ckpt_path = "/work/pi_hzhang2_umass_edu/jnainani_umass_edu/Interp4Robotics/diffusionInterp/data/experiments/low_dim/pusht/diffusion_policy_transformer/train_2/checkpoints/latest.ckpt"
state_dict = torch.load(ckpt_path, map_location=device)
config = state_dict['cfg']
model_config = config['policy']['model']
model_config = {k: v for k, v in model_config.items() if not k.startswith('_target_')}
model = TransformerForDiffusion(**model_config)
noise_scheduler_config = config['policy']['noise_scheduler']
noise_scheduler = DDPMScheduler(**noise_scheduler_config)
policy_params = {
    'model': model,
    'noise_scheduler': noise_scheduler,
    'horizon': config['policy']['horizon'],
    'obs_dim': config['policy']['obs_dim'],
    'action_dim': config['policy']['action_dim'],
    'n_action_steps': config['policy']['n_action_steps'],
    'n_obs_steps': config['policy']['n_obs_steps'],
    'num_inference_steps': config['policy'].get('num_inference_steps', None),
    'obs_as_cond': config['policy'].get('obs_as_cond', False),
    'pred_action_steps_only': config['policy'].get('pred_action_steps_only', False),
}
policy = DiffusionTransformerLowdimPolicy(**policy_params)
policy.load_state_dict(state_dict['state_dicts']['model'])
policy.to(device)

/tmp/ipykernel_2407444/2014934985.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae_weights = torch.load(sae_path)
/tmp/ipykernel_2407444/2014934985.py:37: FutureWarn

DiffusionTransformerLowdimPolicy(
  (model): TransformerForDiffusion(
    (input_emb): Linear(in_features=2, out_features=256, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
    (time_emb): SinusoidalPosEmb()
    (cond_obs_emb): Linear(in_features=20, out_features=256, bias=True)
    (encoder): Sequential(
      (0): Linear(in_features=256, out_features=1024, bias=True)
      (1): Mish()
      (2): Linear(in_features=1024, out_features=256, bias=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-7): 8 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (multihead_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=1024, bias=True)
          (dropout): Dropout(p=

In [ ]:
target_seed = 92400 # int(seed)
# target_step = int(step)
obs_horizon = 2
# target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            # if step_idx == target_step:
            #     target_obs = cond.clone().cpu().numpy()
            #     target_step_image = env.render(mode='rgb_array')
            #     target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

In [2]:
!pip install ipywidgets

In [41]:
target_seed = 92400
latent_idx = 922
steering_coefficient = 0

In [42]:
import ipywidgets as widgets
from IPython.display import display

# Create a mutable container to hold slider and list values
selected_values = {'slider_value': 50, 'integer_list': []}

# Create a slider
slider = widgets.IntSlider(
    value=50,     # Default value
    min=0,        # Minimum value
    max=100,      # Maximum value
    step=1,       # Step size
    description='Value:',
    style={'description_width': 'initial'}  # Adjust label width if needed
)

# Create a SelectMultiple widget for selecting integers
integer_selector = widgets.SelectMultiple(
    options=list(range(0, 101)),  # Range of integers to choose from
    value=[],                     # Default selected values
    description='Select Integers:',
    style={'description_width': 'initial'}
)

# Display the widgets
display(slider, integer_selector)

# Callback for slider value changes
def on_slider_change(change):
    selected_values['slider_value'] = change['new']  # Update the slider value
    print(f"Slider changed to {selected_values['slider_value']}")

# Callback for list selection changes
def on_list_change(change):
    selected_values['integer_list'] = list(change['new'])  # Update the list value
    print(f"Selected integers: {selected_values['integer_list']}")

# Observe changes
slider.observe(on_slider_change, names='value')
integer_selector.observe(on_list_change, names='value')

# Access the values outside the function
print(f"Initial slider value: {selected_values['slider_value']}")
print(f"Initial selected integers: {selected_values['integer_list']}")


IntSlider(value=50, description='Value:', style=SliderStyle(description_width='initial'))

SelectMultiple(description='Select Integers:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, …

Initial slider value: 50
Initial selected integers: []


In [57]:
target_seed = 60600
latent_idx = 43
steering_coefficient = 20

In [58]:
env = PushTKeypointsEnv()
obs_horizon = 2
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, steering_coefficient, latent_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]
                    model_output = policy.model(trajectory, t, cond)
                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{latent_idx}_steer{steering_coefficient}.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{latent_idx}_steer{steering_coefficient}.mp4', embed=True, width=256, height=256)

Seed 60600 Eval: 201it [00:18, 10.86it/s]                         

Reward for 60600:  0.0


In [47]:
Video(f'out/lowdim_seed{target_seed}_f1198_steer0.mp4', embed=True, width=256, height=256)